In [1]:
from Helper.onnx_konv import * 
import json

ModuleNotFoundError: No module named 'onnx'

In [2]:
# Pfad zur JSON mit den Checkpoints
best_checkpoints_json = "/home/jan/studienarbeit/Studienarbeit-CODE_Semantische_Segmentation/FINAL_DATEN/best_checkpoints.json"

# 1) JSON laden
with open(best_checkpoints_json, "r") as f:
    best_checkpoints = json.load(f)

print("Gefundene Modelle in JSON:", list(best_checkpoints.keys()))

calibration_data = "/home/jan/studienarbeit/Studienarbeit-CODE_Semantische_Segmentation/CityscapesDaten/images/"

Gefundene Modelle in JSON: ['lraspp_mobilenet_v3_large', 'fcn_resnet101', 'deeplabv3_resnet50', 'fcn_resnet50', 'deeplabv3_mobilenet_v3_large', 'deeplabv3_resnet101']


In [3]:
# Beispiel: Funktion, die für ein einzelnes Modell den gesamten Konvertierungs- und Testablauf übernimmt.
def convert_and_test_model(checkpoint_path, model_name, conversion_types, calibration_data_path):
    """
    Führt für das angegebene Modell (über den Checkpoint) die gewünschten Konvertierungen durch
    und testet anschließend alle erfolgreich konvertierten Modelle.
    
    :param checkpoint_path: Pfad zum Checkpoint (pickle-Datei)
    :param model_name: Name des Modells
    :param conversion_types: Liste der gewünschten Konvertierungen, z. B.
                             ["fp32", "fp16", "int8_dynamic", "int8_calibrated"]
    :param calibration_data_path: Pfad zu den Kalibrierungsbildern (für INT8)
    """
    print(f"\n=== Starte Konvertierung für Modell: {model_name} ===")
    
    # Erstelle einen Converter für das aktuelle Modell
    converter = ONNXModelConverter(
        checkpoint_path=checkpoint_path,
        model_name=model_name,
        width=2048,
        height=1024,
        input_shape=(1, 3, 520, 520),
        dynamic_batch=True,
        opset_version=13,
        output_dir="./onnx_models",
        skip_local_load=True
    )
    
    # Modell laden
    try:
        converter.load_model()
    except Exception as e:
        print(f"[ERROR] Laden des Modells {model_name} fehlgeschlagen: {e}")
        return  # Abbruch, falls Laden fehlschlägt
    
    # Hier speichern wir die Pfade der erzeugten Modelle, um sie später zu testen
    converted_paths = {}

    # FP32-Export (wird für die meisten Konvertierungen benötigt)
    if "fp32" in conversion_types or any(conv in conversion_types for conv in ["fp16", "int8_dynamic", "int8_calibrated"]):
        try:
            fp32_path = converter.export_fp32()
            converted_paths["fp32"] = fp32_path
        except Exception as e:
            print(f"[ERROR] FP32-Export von {model_name} fehlgeschlagen: {e}")
            # Falls der FP32-Export fehlschlägt, können auch keine weiteren Konvertierungen durchgeführt werden
            return

    # FP16-Konvertierung (sofern erwünscht und verfügbar)
    if "fp16" in conversion_types:
        if HAS_FLOAT16_CONVERTER:
            try:
                fp16_path = converter.convert_fp16(fp32_path)
                converted_paths["fp16"] = fp16_path
            except Exception as e:
                print(f"[WARN] FP16-Konvertierung von {model_name} fehlgeschlagen: {e}")
        else:
            print("[WARN] FP16-Konvertierung nicht verfügbar (float16_converter fehlt).")

    # Dynamische INT8-Quantisierung
    if "int8_dynamic" in conversion_types:
        try:
            int8_dyn_path = converter.quantize_int8_dynamic(fp32_path)
            converted_paths["int8_dynamic"] = int8_dyn_path
        except Exception as e:
            print(f"[ERROR] Dynamische INT8-Quantisierung von {model_name} fehlgeschlagen: {e}")

    # Kalibrierte INT8-Quantisierung
    if "int8_calibrated" in conversion_types:
        try:
            int8_cal_path = converter.quantize_int8_calibrated(
                fp32_onnx_path=fp32_path,
                calibration_data_path=calibration_data_path,
                input_tensor_name="input",
                max_samples=300
            )
            converted_paths["int8_calibrated"] = int8_cal_path
        except Exception as e:
            print(f"[ERROR] Kalibrierte INT8-Quantisierung von {model_name} fehlgeschlagen: {e}")

    # Teste alle erfolgreich konvertierten Modelle
    print(f"\n>>> Teste konvertierte Modelle für {model_name}:")
    for conv, path in converted_paths.items():
        try:
            print(f"{conv.upper()} Modell:")
            converter.test_model(path)
        except Exception as e:
            print(f"[ERROR] Testen des {conv.upper()} Modells von {model_name} fehlgeschlagen: {e}")

    print(f"=== Konvertierung für {model_name} abgeschlossen ===\n")


# # Beispielhafter Aufruf für alle Modelle (best_checkpoints und calibration_data_path müssen definiert sein):
# for model_name, checkpoint_path in best_checkpoints.items():
#     # Hier legst du fest, welche Konvertierung(en) durchgeführt werden sollen.
#     # Beispiel: Nur FP32 und FP16
#     conversions_to_run = ["fp32", "fp16"]
#     # Alternativ kannst du z. B. auch alle vier Varianten angeben:
#     # conversions_to_run = ["fp32", "fp16", "int8_dynamic", "int8_calibrated"]
    
#     convert_and_test_model(checkpoint_path, model_name, conversions_to_run, calibration_data_path)


In [4]:
convert_and_test_model(best_checkpoints["deeplabv3_resnet101"],
                       "deeplabv3_resnet101",
                       ["int8_calibrated"],
                       calibration_data)

convert_and_test_model(best_checkpoints["lraspp_mobilenet_v3_large"],
                       "lraspp_mobilenet_v3_large",
                       ["int8_calibrated"],
                       calibration_data)


=== Starte Konvertierung für Modell: deeplabv3_resnet101 ===
Initialisierung abgeschlossen. Gerät: cuda. Ausgabeordner: ./onnx_models


2025-02-03 19:47:53.937093: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-02-03 19:47:55.273599: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Using CUDA GPU
Model loaded: deeplabv3_resnet101 | Device: cuda 
Skipping local .pth load logic (likely using external Ray checkpoint).
Modell deeplabv3_resnet101 erfolgreich geladen und auf cuda platziert.
FP32 ONNX-Modell exportiert: ./onnx_models/deeplabv3_resnet101_fp32.onnx
Kalibrierungs-Dataloader: Nutze 300 Bilder. CUDA: True
Starte kalibrierte INT8-Quantisierung: ./onnx_models/deeplabv3_resnet101_fp32.onnx


Kalibrierte INT8-Quantisierung abgeschlossen: ./onnx_models/deeplabv3_resnet101_fp32_int8_calibrated.onnx

>>> Teste konvertierte Modelle für deeplabv3_resnet101:
FP32 Modell:
Teste Modell ./onnx_models/deeplabv3_resnet101_fp32.onnx mit Eingabeform (1, 3, 520, 520)
Inferenz erfolgreich. Ausgabeformen:
 - Output 0: shape = (1, 20, 520, 520)
INT8_CALIBRATED Modell:
Teste Modell ./onnx_models/deeplabv3_resnet101_fp32_int8_calibrated.onnx mit Eingabeform (1, 3, 520, 520)
Inferenz erfolgreich. Ausgabeformen:
 - Output 0: shape = (1, 20, 520, 520)
=== Konvertierung für deeplabv3_resnet101 abgeschlossen ===


=== Starte Konvertierung für Modell: lraspp_mobilenet_v3_large ===
Initialisierung abgeschlossen. Gerät: cuda. Ausgabeordner: ./onnx_models
Using CUDA GPU
Model loaded: lraspp_mobilenet_v3_large | Device: cuda 
Skipping local .pth load logic (likely using external Ray checkpoint).
Modell lraspp_mobilenet_v3_large erfolgreich geladen und auf cuda platziert.


FP32 ONNX-Modell exportiert: ./onnx_models/lraspp_mobilenet_v3_large_fp32.onnx
Kalibrierungs-Dataloader: Nutze 300 Bilder. CUDA: True
Starte kalibrierte INT8-Quantisierung: ./onnx_models/lraspp_mobilenet_v3_large_fp32.onnx


Kalibrierte INT8-Quantisierung abgeschlossen: ./onnx_models/lraspp_mobilenet_v3_large_fp32_int8_calibrated.onnx

>>> Teste konvertierte Modelle für lraspp_mobilenet_v3_large:
FP32 Modell:
Teste Modell ./onnx_models/lraspp_mobilenet_v3_large_fp32.onnx mit Eingabeform (1, 3, 520, 520)
Inferenz erfolgreich. Ausgabeformen:
 - Output 0: shape = (1, 20, 520, 520)
INT8_CALIBRATED Modell:
Teste Modell ./onnx_models/lraspp_mobilenet_v3_large_fp32_int8_calibrated.onnx mit Eingabeform (1, 3, 520, 520)
Inferenz erfolgreich. Ausgabeformen:
 - Output 0: shape = (1, 20, 520, 520)
=== Konvertierung für lraspp_mobilenet_v3_large abgeschlossen ===

